In [7]:
import pandas as pd
from tqdm import tqdm

tqdm.pandas()


In [243]:
%run ../paths.py
CONST = CONST()

In [244]:
train = pd.read_csv(CONST.TRAIN_DATA)
train.head()

,Unnamed: 0,tweet,class
0,12761,Mannnnnn these hoes aint loyal,1
1,25141,have a fabulous weekend beautiful souls #nofil...,0
2,53167,#fathersday #dad @user,0
3,17294,RT @SheswantstheD: these hoes ain't loyal http...,1
4,29359,newhairâ¨ð #lapis#rollen#new#hair#brond#gr...,0


In [245]:
train_copy = train.copy()
train_copy.head()

,Unnamed: 0,tweet,class
0,12761,Mannnnnn these hoes aint loyal,1
1,25141,have a fabulous weekend beautiful souls #nofil...,0
2,53167,#fathersday #dad @user,0
3,17294,RT @SheswantstheD: these hoes ain't loyal http...,1
4,29359,newhairâ¨ð #lapis#rollen#new#hair#brond#gr...,0


In [246]:
%run ../src/processing/glove_utils.py

In [247]:
glove_embed = load_embed(CONST.GLOVE_100d)

In [248]:
'<user>' in glove_embed, '<number>' in glove_embed, '<phone_number>' in glove_embed, '<repeat>' in glove_embed,"<allcaps>" in glove_embed

(True, True, False, True, True)

In [312]:
vocab = build_vocab(list(train_copy['tweet'].apply(lambda x:x.split())))
vocab
oov = check_coverage(vocab,glove_embed)
#oov[:10]

100%|██████████| 97055/97055 [00:00<00:00, 228926.20it/s]


Found embeddings for 20.63% of vocab
Found embeddings for  67.83% of all text


In [438]:
add_lower(glove_embed, vocab)

Added 0 words to embedding


In [441]:
#glove_embed['us']

In [315]:
%run ../src/processing/html_tag_remover.py

In [316]:
train_copy['tweet'] = train_copy['tweet'].apply(lambda x : strip_tags(x))

In [337]:
%run ../src/processing/regular_expressions.py

 joke: what do you call a pig with three eyes? <repeat> piiig! <repeat> allcaps <allcaps>  . <repeat>  <number> <number> 


In [338]:
train_copy['tweet'] = train_copy['tweet'].apply(lambda x: tweet_preprocessing(x))


In [320]:
vocab = build_vocab(list(train_copy['tweet'].apply(lambda x:x.split())))
#vocab
oov = check_coverage(vocab,glove_embed)

100%|██████████| 65444/65444 [00:00<00:00, 226897.17it/s]


Found embeddings for 41.40% of vocab
Found embeddings for  86.78% of all text


In [322]:
oov[:10]
#%run ../src/processing/contractions.py

[("i'm", 1946),
 ("don't", 1635),
 ('“', 1234),
 ("can't", 982),
 ("ain't", 712),
 ("you're", 651),
 ('bihday', 584),
 ('bitch.', 509),
 ("y'all", 353),
 ("i'll", 270)]

In [324]:
train_copy['tweet'] = train_copy['tweet'].apply(lambda x: clean_contractions(x, contraction_mapping))
#train_copy.head()


In [325]:
vocab = build_vocab(list(train_copy['tweet'].apply(lambda x:x.split())))
#vocab
oov = check_coverage(vocab,glove_embed)

100%|██████████| 65351/65351 [00:00<00:00, 157328.05it/s]

Found embeddings for 41.44% of vocab
Found embeddings for  88.38% of all text


In [326]:
oov[:10]

[('“', 1234),
 ('bihday', 584),
 ('bitch.', 509),
 ('😂', 224),
 ('day!', 222),
 ('bitch,', 200),
 ('…', 193),
 ('<url>”', 163),
 ('trash.', 155),
 ('😂😂', 145)]

In [263]:
import re
def ap(text):
    text = text.group()
    return text.replace("'","")
re.sub("\w+[']s",ap,"'s  'father's,mother's,brother's,sister's,baby's father's also uncle")

"'s  'fathers,mothers,brothers,sisters,babys fathers also uncle"

In [327]:
%run ../src/processing/spellings.py

In [328]:
train_copy['tweet'] = train_copy['tweet'].apply(lambda x: replace_elongated_text(x))


In [329]:
vocab = build_vocab(list(train_copy['tweet'].apply(lambda x:x.split())))
#vocab
oov = check_coverage(vocab,glove_embed)

100%|██████████| 64895/64895 [00:00<00:00, 168517.12it/s]


Found embeddings for 41.80% of vocab
Found embeddings for  88.46% of all text


In [330]:
oov[:10]

[('“', 1234),
 ('bihday', 584),
 ('bitch.', 509),
 ('😂😂', 379),
 ('😂', 224),
 ('day!', 222),
 ('bitch,', 200),
 ('…', 193),
 ('<url>”', 163),
 ('trash.', 155)]

In [331]:
%run ../src/processing/special_char.py

In [333]:
train_copy['tweet'] = train_copy['tweet'].apply(lambda x: clean_special_chars(x,punct,punct_mapping))


In [339]:
vocab = build_vocab(list(train_copy['tweet'].apply(lambda x:x.split())))
#vocab
oov = check_coverage(vocab,glove_embed)

100%|██████████| 47082/47082 [00:00<00:00, 196901.27it/s]


Found embeddings for 62.03% of vocab
Found embeddings for  96.45% of all text


In [340]:
oov[:10]

[('bihday', 636),
 ('😂😂', 447),
 ('😂', 270),
 ('â\x9d¤ï¸\x8f', 122),
 ('★', 121),
 ('ð\x9f\x98\x8a', 116),
 ('ð\x9f\x98\x8d', 103),
 ('ð\x9f\x98', 86),
 ('😒', 82),
 ('ð\x9f\x98\x82', 73)]

In [341]:
%run ../src/processing/emojis.py

smiling_face fmgvmn steaming_bowl  sleeping_face  exploding_head  woozy_face fgfb thinking_face


In [398]:
#train_copy['removed_contr'][len('removed_contr')<0]
#train_copy.head()
drop_rows = train_copy[train_copy['tweet']==""].tweet
train_copy.drop(drop_rows.index, axis=0,inplace=True)
#train_copy.drop([[1859]],inplace=True)
train_copy['tweet'] = train_copy['tweet'].apply(lambda x: remove_duplicates(x) )


In [399]:
vocab = build_vocab(list(train_copy['tweet'].apply(lambda x:x.split())))
#vocab
oov = check_coverage(vocab,glove_embed)


100%|██████████| 46972/46972 [00:00<00:00, 192029.38it/s]

Found embeddings for 62.18% of vocab
Found embeddings for  96.45% of all text


In [400]:
oov[:10]

[('😂', 717),
 ('bihday', 636),
 ('â\x9d¤ï¸\x8f', 122),
 ('★', 121),
 ('ð\x9f\x98\x8a', 116),
 ('ð\x9f\x98\x8d', 103),
 ('😒', 90),
 ('😭', 88),
 ('ð\x9f\x98', 86),
 ('ð\x9f\x98\x82', 73)]

In [401]:
import string
white_list = string.ascii_letters + string.digits + ' '
white_list += "'"
#oov[:100]

In [402]:
glove_chars = ''.join([c for c in tqdm(glove_embed) if len(c) == 1])
glove_symbols = ''.join([c for c in glove_chars if not c in white_list])
glove_symbols


100%|██████████| 1193534/1193534 [00:02<00:00, 502976.78it/s]


'.:,!"?()-/*>^<&_~;|[]`$=+%@\\#}{'

In [442]:
all_characters = build_vocab(list(train_copy["tweet"]))
all_characters
#symbols_to_delete = ''.join([c for c in all_characters if not c in white_list if not c in glove_symbols])
#symbols_to_delete

100%|██████████| 42348/42348 [00:01<00:00, 21843.21it/s]


{'m': 75657,
 'a': 221702,
 'n': 157365,
 ' ': 758631,
 't': 212200,
 'h': 119004,
 'e': 307542,
 's': 190447,
 'o': 192506,
 'i': 175152,
 'l': 128556,
 'y': 70983,
 'v': 23871,
 'f': 50590,
 'b': 60077,
 'u': 116030,
 'w': 49263,
 'k': 33656,
 'd': 81911,
 'r': 171964,
 '<': 58916,
 '>': 59127,
 'c': 77053,
 'p': 64429,
 ':': 9301,
 '#': 1901,
 'g': 63190,
 'j': 6764,
 '.': 23459,
 ',': 9280,
 "'": 1712,
 '!': 9008,
 '-': 3419,
 '"': 7880,
 '/': 917,
 'z': 3208,
 'x': 4276,
 '?': 3293,
 'q': 1644,
 '&': 2019,
 '~': 137,
 '*': 528,
 '=': 154,
 '@': 639,
 '$': 214,
 ';': 156,
 '%': 246,
 '+': 156,
 '(': 584,
 '|': 237,
 ')': 570,
 '^': 38,
 '{': 7,
 '}': 6,
 '\\': 14,
 '[': 5,
 ']': 4}

In [404]:
remove_dict = {ord(c):f'' for c in symbols_to_delete}


def handle_punctuation(x):
    x = x.translate(remove_dict)
    return x


In [405]:
train_copy['tweet'] = train_copy['tweet'].apply(lambda x: handle_punctuation(x))


In [406]:
vocab = build_vocab(list(train_copy['tweet'].apply(lambda x:x.split())))
#vocab
oov = check_coverage(vocab,glove_embed)

100%|██████████| 40761/40761 [00:00<00:00, 142042.24it/s]


Found embeddings for 72.19% of vocab
Found embeddings for  98.05% of all text


In [408]:
oov[:100]

[('bihday', 660),
 ('prayfororlando', 74),
 ('allahsoil', 63),
 ('erience', 55),
 ('staing', 54),
 ('impoant', 49),
 ('brexit', 47),
 ('orlandoshooting', 45),
 ('staed', 44),
 ('inshot', 39),
 ('spos', 38),
 ('heabroken', 35),
 ('ripchristina', 34),
 ('altwaystoheal', 30),
 ('udtapunjab', 30),
 ('melancholymusic', 25),
 ('neverump', 24),
 ('ripchristinagrimmie', 24),
 ('summeime', 23),
 ('ected', 23),
 ('bouncingbaby', 22),
 ('suppoing', 20),
 ('unfounately', 20),
 ('suppoers', 19),
 ('ukhx', 19),
 ('staup', 19),
 ('ectations', 19),
 ('christinagrimmie', 18),
 ('heabreaking', 18),
 ('fuher', 17),
 ('bihdaygirl', 17),
 ('wednesdaywisdom', 17),
 ('ensive', 17),
 ('teambts', 17),
 ('friyay', 16),
 ('blacklivesmatter', 16),
 ('paner', 16),
 ('osed', 15),
 ('enteainment', 15),
 ('toptags', 14),
 ('oppounity', 14),
 ('aicle', 13),
 ('muzzie', 13),
 ('altright', 13),
 ('quotestags', 12),
 ('orlandonightclubshooting', 12),
 ('loveisland', 12),
 ('bihdays', 11),
 ('notmypresident', 11),
 ('anto

In [416]:
vocab

{'mann': 13,
 'these': 1263,
 'hoes': 1944,
 'aint': 198,
 'loyal': 107,
 'have': 2589,
 'a': 12388,
 'fabulous': 27,
 'weekend': 510,
 'beautiful': 402,
 'souls': 12,
 'nofilter': 33,
 'fathersday': 325,
 'dad': 281,
 '<': 57755,
 'user': 28328,
 '>': 57966,
 'rt': 5985,
 'allcaps': 9242,
 ':': 9301,
 'is': 6287,
 'not': 5465,
 'url': 2389,
 'newhair': 4,
 'lapis': 2,
 '#': 1901,
 'rollen': 2,
 'new': 1012,
 'hair': 221,
 'brond': 1,
 'gradation': 1,
 'my': 5592,
 'instapic': 19,
 'instagram': 103,
 'instagood': 140,
 'l': 139,
 'number': 8515,
 'girl': 635,
 'g': 89,
 'this': 3761,
 'just': 2234,
 'in': 5614,
 'do': 3089,
 'trust': 132,
 'bitches': 2449,
 'need': 696,
 'guidance': 10,
 '.': 23459,
 'hey': 156,
 'heres': 46,
 'someone': 314,
 'else': 118,
 'who': 986,
 'suppos': 9,
 'you': 9667,
 'are': 3279,
 'lifestyle': 75,
 'also': 130,
 ',': 9280,
 'happy': 1380,
 'fathers': 571,
 'day': 2053,
 'i': 12688,
 'am': 2963,
 'thankful': 428,
 'for': 4889,
 'socks': 20,
 'positive': 44